In [20]:
import openmmtools
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
import numpy as np
import yank
from yank.restraints import RestraintState
from yank.experiment import Topography
from importlib import reload

In [21]:
pdb = app.PDBFile('tip3p.pdb')
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, 
    nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds, rigidWater=True, 
    ewaldErrorTolerance=0.0005)

integrator = mm.LangevinIntegrator(100*unit.kelvin, 1.0/unit.picoseconds, 
    2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

In [22]:
reload(openmmtools.alchemy)
factory = openmmtools.alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False, split_alchemical_forces = True, alchemical_pme_treatment = 'exact')
reference_system = system
alchemical_region_1 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [3,4,5], name='one')
alchemical_region_0 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [0,1,2], name='zero')
alchemical_region_2 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [6,7,8], name='two')

alchemical_system_in = factory.create_alchemical_system(reference_system, alchemical_regions = [alchemical_region_0, alchemical_region_1, alchemical_region_2])#, alchemical_region3], )
alchemical_state_zero = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'zero')
alchemical_state_one = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'one')
alchemical_state_two = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'two')

Using 3 alchemical regions
[[0, 1, 2], [3, 4, 5], [6, 7, 8]]
HarmonicBondForce
HarmonicBondForce
HarmonicBondForce
HarmonicAngleForce
HarmonicAngleForce
HarmonicAngleForce
PeriodicTorsionForce
PeriodicTorsionForce
PeriodicTorsionForce
NonbondedForce
zero
[0, 1, 2, 3, 4, 5, 6, 7, 8]
NonbondedForce
one
[0, 1, 2, 3, 4, 5, 6, 7, 8]
NonbondedForce
two
[0, 1, 2, 3, 4, 5, 6, 7, 8]
CMMotionRemover
CMMotionRemover
CMMotionRemover
 [<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f808bb40330> >, <simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f808bb40f90> >, <simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f808bb40de0> >, <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f808bb40810> >, <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::Har

In [23]:
#restraint_state = RestraintState(lambda_restraints=1.0)
#print(restraint_state)

In [24]:
TS = openmmtools.states.ThermodynamicState(alchemical_system_in, temperature=300*unit.kelvin, pressure=1*unit.bar)
composable_states = [alchemical_state_zero, alchemical_state_one, alchemical_state_two]#, restraint_state]
compound_state = openmmtools.states.CompoundThermodynamicState(
            thermodynamic_state=TS, composable_states=composable_states)

{'lambda_electrostatics_one': None, 'lambda_sterics_two': None, 'lambda_sterics_zero': 1.0, 'lambda_electrostatics_two': None, 'lambda_sterics_one': None, 'lambda_electrostatics_zero': 1.0, 'lambda_bonds': None, 'lambda_torsions': None, 'lambda_angles': None}
{'lambda_electrostatics_one': 1.0, 'lambda_sterics_two': None, 'lambda_sterics_zero': None, 'lambda_electrostatics_two': None, 'lambda_sterics_one': 1.0, 'lambda_electrostatics_zero': None, 'lambda_bonds': None, 'lambda_torsions': None, 'lambda_angles': None}
{'lambda_electrostatics_one': None, 'lambda_sterics_two': 1.0, 'lambda_sterics_zero': None, 'lambda_electrostatics_two': 1.0, 'lambda_sterics_one': None, 'lambda_electrostatics_zero': None, 'lambda_bonds': None, 'lambda_torsions': None, 'lambda_angles': None}
lambda_electrostatics_one None
lambda_sterics_two None
lambda_sterics_zero 1.0
lambda_electrostatics_two None
lambda_sterics_one None
lambda_electrostatics_zero 1.0
lambda_bonds None
lambda_torsions None
lambda_angles No

In [27]:
reload(openmmtools.alchemy)
integrator = mm.VerletIntegrator(1.0*unit.femtosecond)
compound_state._composable_states[0].lambda_sterics_zero = 0.0
compound_state._composable_states[0].lambda_electrostatics_zero = 0.0
compound_state._composable_states[1].lambda_sterics_one = 1.0
compound_state._composable_states[1].lambda_electrostatics_one = 1.0
compound_state._composable_states[2].lambda_sterics_two = 0.0
compound_state._composable_states[2].lambda_electrostatics_two = 1.0
context = compound_state.create_context(integrator)

Apply to System
zero lambda_electrostatics_one
zero lambda_electrostatics_one
zero lambda_electrostatics_one
zero lambda_electrostatics_two
zero lambda_electrostatics_two
zero lambda_electrostatics_two
zero lambda_electrostatics_zero
lambda_electrostatics_zero
1.0 True
zero lambda_electrostatics_zero
lambda_electrostatics_zero
1.0 True
zero lambda_electrostatics_zero
lambda_electrostatics_zero
1.0 True
zero lambda_sterics_one
zero lambda_sterics_one
zero lambda_sterics_two
zero lambda_sterics_two
zero lambda_sterics_zero
lambda_sterics_zero
zero lambda_sterics_zero
lambda_sterics_zero
True
SET PME
-0.0 0.0 -0.834
0.0 0.0 0.417
0.0 0.0 0.417
Apply to System
one lambda_electrostatics_one
lambda_electrostatics_one
1.0 False
one lambda_electrostatics_one
lambda_electrostatics_one
1.0 False
one lambda_electrostatics_one
lambda_electrostatics_one
1.0 False
one lambda_electrostatics_two
one lambda_electrostatics_two
one lambda_electrostatics_two
one lambda_electrostatics_zero
one lambda_elect

In [28]:
context.setPositions(pdb.positions)
for param in context.getParameters():
    print(param)
    #print(context.getParameter(param))

integrator.step(1000)
print('No Error')

AndersenCollisionFrequency
AndersenTemperature
MonteCarloPressure
MonteCarloTemperature
_update_alchemical_charges
lambda_electrostatics_one
lambda_electrostatics_two
lambda_electrostatics_zero
lambda_sterics_one
lambda_sterics_two
lambda_sterics_zero
softcore_a
softcore_alpha
softcore_b
softcore_beta
softcore_c
softcore_d
softcore_e
softcore_f


Exception: Particle coordinate is nan